<a href="https://colab.research.google.com/github/ikram3008/Text-Summarization-Module-Agile-Loop-/blob/main/Text_Summarization_Module_(Agile_Loop).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Required Libraries
!pip install transformers gtts

In [2]:
# Import Libraries
from transformers import pipeline
from gtts import gTTS
from IPython.display import Audio

In [22]:
class TextSummarizer:
    def __init__(self):
        # Load summarization and sentiment analysis models
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        self.sentiment_analyzer = pipeline("sentiment-analysis")

    def summarize(self, text, summary_type="abstractive", sentiment_filter=None, max_length=50, output_format="text", voice_narration=False):
        """
        Summarize text with options for extractive/abstractive, sentiment filtering, and output formats.
        """
        # Filter by sentiment (if requested)
        if sentiment_filter:
            text = self._filter_by_sentiment(text, sentiment_filter)

        # Generate summary
        if summary_type == "abstractive":
            summary = self._abstractive_summarize(text, max_length)
        else:
            summary = self._extractive_summarize(text, max_length)

        # Format output
        result = self._format_output(summary, output_format)

        # Add voice narration (if requested)
        if voice_narration:
            self._text_to_speech(summary)

        return result

    def _filter_by_sentiment(self, text, target_sentiment):
        """
        Filter sentences by sentiment (positive/negative).
        """
        sentences = text.split(". ")
        filtered_sentences = [s for s in sentences if self.sentiment_analyzer(s)[0]["label"].lower() == target_sentiment.lower()]
        return ". ".join(filtered_sentences)

    def _abstractive_summarize(self, text, max_length):
        """
        Generate abstractive summary using BART model.
        """
        return self.summarizer(text, max_length=max_length, min_length=25, do_sample=False)[0]["summary_text"]

    def _extractive_summarize(self, text, max_length):
        """
        Simple extractive summary by selecting the first few sentences.
        """
        sentences = text.split(". ")
        return ". ".join(sentences[:max_length // 20])  # Adjust based on sentence length

    def _format_output(self, summary, output_format):
        """
        Format the output as text, JSON, or Markdown.
        """
        if output_format == "json":
            return {"summary": summary}
        elif output_format == "markdown":
            return f"**Summary:**\n\n{summary}"
        else:
            return summary

    def _text_to_speech(self, text):
        """
        Convert text to speech using gTTS.
        """
        tts = gTTS(text=text, lang="en")
        tts.save("summary.mp3")
        return Audio("summary.mp3", autoplay=True)

# Example Usage
if __name__ == "__main__":
    # Sample text
    sample_text = """
    Users are reporting mixed experiences with the new update. Many appreciate the improved interface,
    but several have encountered performance issues. The AI features seem to work well for most users,
    though a small group complains about increased battery usage.
    """

    # Initialize summarizer
    summarizer = TextSummarizer()

    # Example 1: Abstractive summary with sentiment filtering
    print("Negative Aspects Summary:")
    print(summarizer.summarize(
        sample_text,
        sentiment_filter="negative",
        output_format="markdown"
    ))

    # Example 2: Extractive summary with voice narration
    print("\nGeneral Summary:")
    result = summarizer.summarize(
        sample_text,
        summary_type="extractive",
        output_format="text",
        voice_narration=True
    )
    print(result)

Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Negative Aspects Summary:


Your max_length is set to 50, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


**Summary:**

Users are reporting mixed experiences with the new update. Many appreciate the improved interface, but several have encountered performance issues.

General Summary:

    Users are reporting mixed experiences with the new update. Many appreciate the improved interface, 
    but several have encountered performance issues


In [31]:
# Feature 1: Abstractive Summarization


text = """Artificial intelligence is transforming industries across the globe. From healthcare to finance,
AI is being used to automate tasks, improve decision-making, and enhance customer experiences.
However, there are concerns about job displacement and ethical implications of AI technologies."""

summary = summarizer.summarize(text, summary_type="abstractive", max_length=50)
print(summary)

AI is being used to automate tasks, improve decision-making, and enhance customer experiences. There are concerns about job displacement and ethical implications of AI technologies.


In [32]:
# Feature 2: Extractive Summarization

text = """
The new smartphone model has received positive reviews for its camera quality and battery life.
However, some users have reported issues with the display brightness and software glitches.
Overall, the phone is considered a good value for its price.
"""

summary = summarizer.summarize(text, summary_type="extractive", max_length=50)
print(summary)



The new smartphone model has received positive reviews for its camera quality and battery life. 
However, some users have reported issues with the display brightness and software glitches


In [34]:
# Feature 3: Sentiment Filtering

text = """
Customers love the new design and user-friendly interface. However, many are disappointed with the battery life.
Some users also reported that the app crashes frequently. The customer support team has been praised for their quick responses.
"""

summary = summarizer.summarize(text, sentiment_filter="negative")
print(summary)


Your max_length is set to 50, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


Some users also reported that the app crashes frequently. However, many are disappointed with the battery life of the phone.


In [35]:
# Feature 4: Multi-Format Output

# JSON output
summary = summarizer.summarize(text, output_format="json")
print(summary)

# Markdown output
summary = summarizer.summarize(text, output_format="markdown")
print(summary)



{'summary': 'Customers love the new design and user-friendly interface. However, many are disappointed with the battery life. The customer support team has been praised for their quick responses.'}
**Summary:**

Customers love the new design and user-friendly interface. However, many are disappointed with the battery life. The customer support team has been praised for their quick responses.


In [36]:
# Feature 5: Voice Narration


summary = summarizer.summarize(text, voice_narration=True)
print(summary)

Customers love the new design and user-friendly interface. However, many are disappointed with the battery life. The customer support team has been praised for their quick responses.
